In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 54.4 MB/s eta 0:00:00


In [ ]:

from google.colab import drive
import os
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


#  Importer les bibliothèques nécessaires





In [ ]:
import os
import pdfplumber
import pandas as pd
import re



# Extraire Titre et Texte (Sujet de divers et Articles de loi)





In [ ]:
# Supprimer des mots ne souhaites pas
def remove_unwanted_terms(text):
    unwanted_terms = [
        'Mme la présidente.',
        'Mme le présidente.',
        'M. le président.',
        'Mme le président.',
        'Mme la président.'
    ]
    for term in unwanted_terms:
        text = text.replace(term, '')
    return text.strip()

"""
Cette fonction va : lire le fichier PDF, parcourir chaque page pour extraire les titres (Titre) et le texte (Texte)
Titre : Filtrera les parties en gras car le titre est en gras
Texte : Recherchera les sections en gras avec les mots suivants : Article, Avant l’article, Après l’article
"""
def extract_Titre_Texte(pdf_path):
    data = []
    expected_number = 1  # Variable pour suivre le numéro d'ordre actuel
    with pdfplumber.open(pdf_path) as pdf:
        for page_number in range(len(pdf.pages)):
            current_line = ""
            page = pdf.pages[page_number]
            chars = page.chars  # chars là một cái dictionary # chars est un dictionnaire

            for char in chars:
                if "Bold" in char["fontname"]: # Si "Bold" est présent dans le nom de la police -> C'est une section en gras

                    current_line += char["text"] # Le texte ici est composé de caractères séparés (a b c 1 2 -)
                    # Si ce n'est pas en gras et que ce n'est pas vide -> La section en gras a été complètement récupérée
                     # current_line devient une phrase complète en gras
                elif current_line.strip():
                    match = re.match(r"(\d+)\s+([A-Za-zÀ-Ýa-zà-ý\s:’.,;?!()\[\]«»\-]*)(?:\s*(Mme la présidente\.|Mme le présidente\.|M\. le président\.|Mme le président\.|Mme la président\.))?$", current_line.strip())
                    if match and int(match.group(1)) == expected_number:
                        # Si le premier numéro correspond au numéro d'ordre attendu
                        expected_number += 1  # Tăng số thứ tự cho dòng tiếp theo # Incrémenter le numéro d'ordre pour la ligne suivante
                        title_text = f"{match.group(2).strip()}"
                        data.append((page_number + 1, current_line.strip(), title_text, ""))
                    else:
                        data.append((page_number + 1, current_line.strip(), None, ""))
                    current_line = ""

        # Après la boucle for, traiter la dernière ligne de la page (souvent le numéro de page)
            if current_line.strip():
                # 1 ou plusieurs chiffres (\d+), (1 ou plusieurs espaces (\s+)) (groupe 1)
                # ... (groupe 2)
                match = re.match(r"(\d+)\s+([A-Za-zÀ-Ýa-zà-ý\s:’.,;?!()\[\]«»\-]*)(?:\s*(Mme la présidente\.|Mme le présidente\.|M\. le président\.|Mme le président\.|Mme la président\.))?$", current_line.strip())
                if match and int(match.group(1)) == expected_number:
                    expected_number += 1
                    title_text = f"{match.group(2).strip()} "
                    data.append((page_number + 1, current_line.strip(), title_text, ""))
                else:
                    data.append((page_number + 1, current_line.strip(), None, ""))

    # Supprimer les termes indésirables à la fin du texte (Mme la présidente) du TITRE
        for i in range(len(data)):
            if data[i][2] is not None:
                data[i] = (data[i][0], data[i][1], remove_unwanted_terms(data[i][2]), data[i][3])

# Remplacer les valeurs None ou vides par la valeur du Titre précédent, et les modifier si un autre Titre est trouvé
    last_titre_text = None
    for i in range(len(data)):
        if data[i][2] is not None:
            last_titre_text = data[i][2]
        elif last_titre_text is not None:
            data[i] = (data[i][0], data[i][1], last_titre_text, data[i][3])

  # Si les sections en gras contiennent des termes comme : Après l’article, Article, Avant l’article, les ajouter à la liste (appelée thème)
    for i in range(len(data)):
        if data[i][1] and ("Après l’article" in data[i][1] or "Article" in data[i][1] or "Avant l’article" in data[i][1]):
            data[i] = (data[i][0], data[i][1], data[i][2], data[i][1])


  # Remplacer les valeurs vides ou None par la valeur du Texte précédent, et les modifier si un autre Texte est trouvé
    last_texte_text = ""
    for i in range(len(data)):
        if data[i][3] != "":
            last_texte_text = data[i][3]
        else:
            data[i] = (data[i][0], data[i][1], data[i][2], last_texte_text)

    return data


# Extraire Thème ( Nature de textes de loi)

In [ ]:

'''
# Cette fonction va traiter le thème (Nature de texte) : Ce sera des sections de texte en majuscules mais non en gras, contenant les deux termes suivants : "PROPOSITION DE LOI" ou "PROJET DE LOI"
'''
def extract_topic(pdf_path, data):
    topic_lines = []

    with pdfplumber.open(pdf_path) as pdf:
        for page_number in range(len(pdf.pages)):
            current_line = ""
            page = pdf.pages[page_number]
            chars = page.chars

            for char in chars:
                if "Bold" not in char["fontname"]:
                    current_line += char["text"]

            # Verifier pour mot clés "PROPOSITION DE LOI" ou "PROJET DE LOI"
            if "PROPOSITION DE LOI" in current_line or "PROJET DE LOI" in current_line:
                topic_lines.append((page_number + 1, current_line))

    theme_lines = {}
    for page, text in topic_lines:
        # Regex pour "PROPOSITION DE LOI"
        # PROPOSITION DE LOI ne doit être attaché à aucun autre mot
        pattern_proposition = r'\b[A-ZÀ-Ý\s\.,;’!?()_-]*PROPOSITION DE LOI[A-ZÀ-Ý\s\.,;’!?()_-]*\b'
        matches_proposition = re.findall(pattern_proposition, text, re.DOTALL) # Trouver toutes les chaînes correspondantes, y compris les cas avec \n, avec re.

        # Regex pour "PROJET DE LOI"
        pattern_projet = r'\b[A-ZÀ-Ý\s\.,;’!?()_-]*PROJET DE LOI[A-ZÀ-Ý\s\.,;’!?()_-]*\b'
        matches_projet = re.findall(pattern_projet, text, re.DOTALL)

        # Combine results # Combiner les résultats
        matches = matches_proposition + matches_projet

        if matches:
            for match in matches:
                 # Remplacer tous les espaces multiples par un seul espace et supprimer les espaces en début et en fin de chaîne
                cleaned_match = re.sub(r'\s+', ' ', match.strip())
                #  supprimer les caractères spéciaux au début et à la fin

                # ^ au début, $ à la fin
                cleaned_match = re.sub(r'^[\s\.,;’!?()_-]+|[\s\.,;’!?()_-]+$', '', cleaned_match)
                words = cleaned_match.split()

                # Si le mot existe et que le dernier élément a une longueur < 2 : alors supprimer
                while words and len(words[-1]) < 2:
                    words.pop()
                cleaned_match = ' '.join(words)
                theme_lines[page] = cleaned_match


    # Ajouter Thème à bold_lines pour chaque page
    for i in range(len(data)):
        page_number = data[i][0]   # Si page_number correspond -> transférer
        if page_number in theme_lines:
            data[i] = (data[i][0], data[i][1], data[i][2], data[i][3], theme_lines[page_number])

# Remplacer les valeurs vides par la valeur du thème précédent
    last_topic_text = ""
    for i in range(len(data)):
        if len(data[i]) > 4 and data[i][4] != "":
            last_topic_text = data[i][4]
        else:
            data[i] = (data[i][0], data[i][1], data[i][2], data[i][3], last_topic_text)

    return data


# Extraire nom des intervenants

In [ ]:

def extract_nom_intervenants(data):
    # Préfixe
    prefixes = ['Mme le', 'Mme la', 'M. le', 'M.', 'Mme']
    extracted_nom_intervenants = []
    page_values = {}

    for page_number, text,titre, texte, topic in data:
    # Vérifier si le texte est un nombre, si c'est un nombre -> numéro de page
        if text.isdigit():
            page_values[page_number] = text  # dic ( key: page_number)-> values : numéro de page

    for page_number, text, titre, texte, topic in data:
        for prefix in prefixes:
            if prefix in text:
                start_index = text.find(prefix)   # Trouver l'index commence
                end_index = start_index + len(prefix) # Trouver l'index fin de préfixe
                for i in range(end_index, len(text)):
                    if text[i] in ['.', ',']:
                        end_index = i + 1  # l'index fin de nom
                        break
                page_value = page_values.get(page_number, "") # extraire values pour page_number(key)
                extracted_nom_intervenants.append((text[start_index:end_index].strip(), page_value, titre, texte, topic))
                break

    return extracted_nom_intervenants

# Appeler et traiter en tant que DataFrame


In [ ]:
def process_file(pdf_path,date):

    data = extract_Titre_Texte(pdf_path)
    data = extract_topic(pdf_path, data)
    data = extract_nom_intervenants(data)
    df = pd.DataFrame(data, columns=['Nom_intervenants', 'Num_page','Titre','Texte','Thème'])
    if not df.empty:
        # Trouver l'index correspondant aux noms
        first_president_row = df[(df['Nom_intervenants'] == 'Mme la présidente.') |
                                 (df['Nom_intervenants'] == 'Mme le présidente.') |
                                 (df['Nom_intervenants'] == 'M. le président.') |
                                 (df['Nom_intervenants'] == 'Mme le président.') |
                                 (df['Nom_intervenants'] == 'Mme la président.')].index[0]  # .index: renvoie les indices des lignes qui satisfont au moins une des conditions spécifiées.
        # Prendre à partir de cet index jusqu'à la fin
        df = df[first_president_row:]
        df['Texte'] = df['Texte'].apply(remove_unwanted_terms)
        df['Numero'] = range(1, len(df) + 1)
        df['Date_seances'] = date
        df = df[['Numero', 'Nom_intervenants', 'Date_seances','Num_page','Titre','Texte','Thème']]

    return df


# Extraire les informations de date, appeler la fonction pour traiter les données et les enregistrer dans un fichier CSV


In [ ]:
data_dir = "/content/drive/MyDrive/Tieng_Phap"
output_dir = "/content/drive/MyDrive/French_output_date_npage_part13"
os.makedirs(output_dir, exist_ok=True)

pdf_files = [f for f in os.listdir(data_dir) if f.endswith('.pdf')]
french_months = ["janvier", "février", "mars", "avril", "mai", "juin", "juillet", "août", "septembre", "octobre", "novembre", "décembre"]

for pdf_file in pdf_files:
    pdf_path = os.path.join(data_dir, pdf_file)
    filename = os.path.splitext(pdf_file)[0]
    year, month, day = filename[1:5], filename[5:7], filename[7:9]
    month_name = french_months[int(month) - 1]  # Convertir mois de numero à mois en français
    date = f"{day} {month_name} {year}"
    df = process_pdf_file(pdf_path, date)

    if not df.empty:
        csv_file_name = f"{filename}.csv"
        csv_file_path = os.path.join(output_dir, csv_file_name)
        df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')
        print(f"Processed and saved: {csv_file_name}")



# Fusionner les fichiers par date -> Un fichier unique:

In [ ]:
import os
import pandas as pd

# Definier les chemins
input_dir = "/content/drive/MyDrive/French_output_date_npage_part13"
output_folder = '/content/drive/MyDrive/French_Outpu_22_9/data'
output_file = 'combined_df.csv'
os.makedirs(output_folder, exist_ok=True)

# Obtenir la liste de tous les fichiers CSV dans le répertoire
csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# Créer une liste pour stocker les DataFrames
dfs = []

# Lire et fusionner les fichiers CSV
for file in csv_files:
    file_path = os.path.join(input_dir, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
combined_df.to_csv(f'{output_folder}/{output_file}', index=False,encoding='utf-8-sig')


# Vérifier s'il y a des noms incorrects





In [ ]:
import re
target_strings = ['Mme le', 'Mme la', 'M. le', 'M.', 'Mme']

# Créer un modèle regex pour correspondre exactement aux caractères dans la liste
pattern = r'^(' + '|'.join(map(re.escape, target_strings)) + r')$'

# Lọc các hàng trong DataFrame
filtered_df = combined_df[combined_df['Nom_intervenants'].str.match(pattern)]
filtered_df


# Filtrer par nom


In [ ]:
import pandas as pd
import re
import os

# fichier combined est fichier groupé tous les files qui classé par date de séances
combined_df = pd.read_csv("/content/drive/MyDrive/French_Outpu_22_9/data/combined_df.csv")
output_dir = "/content/drive/MyDrive/French_Outpu_22_9/original_data"

os.makedirs(output_dir, exist_ok=True)

# Supprimer les virgules ou les points à la fin de 'Nom_intervenants'
combined_df['Nom_intervenants'] = combined_df['Nom_intervenants'].str.rstrip(',.')

# Supprimer les éléments dans 'unwanted_names'
unwanted_names = ['Mme le présidente', 'M. le président', 'Mme le président', 'Mme la président','Mme le présidente.', 'M. le président.', 'Mme le président.', 'Mme la président','Mme la présidente']
combined_df = combined_df[~combined_df['Nom_intervenants'].isin(unwanted_names)]

# Extraire chaque élément un par un
unique_names = combined_df['Nom_intervenants'].unique()
print(unique_names.shape)

for name in unique_names:
    # Filter par nom de chaque Sénateur
    filtered_data = combined_df[combined_df['Nom_intervenants'] == name][['Date_seances', 'Num_page,'Titre','Texte','Thème']]

    # Supprimer les lignes en double en se basant sur les colonnes 'Date_seances' et 'Num_page'
    filtered_data = filtered_data.drop_duplicates(subset=['Date_seances', 'Num_page'])

    # Supprimer 'Mme.' et 'M.' et remplacer les espaces par des underscores
    formatted_name = name.replace("M.", "").replace("Mmes", "").replace("Mme", "").replace(" ", "_")
    if formatted_name.startswith("_"): # Mme Sophie Martin -> _Sophie_Martin
        formatted_name = formatted_name[1:]

    filename = f"{formatted_name}.csv"
    file_path = os.path.join(output_dir, filename)

    # Ajouter un suffixe
    if os.path.exists(file_path):
        base, ext = os.path.splitext(filename)
        duplicate_filename = f"{base}_Rapporteur{ext}"
        file_path = os.path.join(output_dir, duplicate_filename)

    filtered_data.to_csv(file_path, index=False, encoding='utf-8-sig')



(456,)


# Filtre les noms par liste de Senateurs 2022

In [ ]:

formatted_names = [
    "Pascal_Allizard",
    "Jean-Claude_Anglars",
    "Maurice_Antiste",
    "Cathy_Apourceau-Poly",
    "Jean-Michel_Arnaud",
    "Stéphane_Artano",
    "Viviane_Artigalas",
    "Éliane_Assassi",
    "David_Assouline",
    "Serge_Babary",
    "Jérémy_Bacchi",
    "Jean_Bacci",
    "Jean-Pierre_Bansard",
    "Julien_Bargeton",
    "Philippe_Bas",
    "Jérôme_Bascher",
    "Arnaud_Bazin",
    "Arnaud_de_Belenet",
    "Bruno_Belin",
    "Nadine_Bellurot",
    "Catherine_Belrhiti",
    "Guy_Benarroche",
    "Esther_Benbassa",
    "Martine_Berthet",
    "Joël_Bigot",
    "Christian_Bilhac",
    "Annick_Billon",
    "Étienne_Blanc",
    "Jean-Baptiste_Blanc",
    "Florence_Blatrix_Contat",
    "Éric_Bocquet",
    "Christine_Bonfanti-Dossat",
    "François_Bonhomme",
    "Bernard_Bonne",
    "François_Bonneau",
    "Philippe_Bonnecarrère",
    "Nicole_Bonnefoy",
    "Michel_Bonnus",
    "Alexandra_Borchio_Fontimp",
    "Denis_Bouad",
    "Gilbert_Bouchet",
    "Céline_Boulay-Espéronnier",
    "Yves_Bouloux",
    "Hussein_Bourgi",
    "Toine_Bourrat",
    "Jean-Marc_Boyer",
    "Valérie_Boyer",
    "Daniel_Breuiller",
    "Isabelle_Briquet",
    "Max_Brisson",
    "Céline_Brulin",
    "François-Noël_Buffet",
    "Bernard_Buis",
    "Laurent_Burgoa",
    "Henri_Cabanel",
    "Alain_Cadec",
    "Olivier_Cadic",
    "François_Calvet",
    "Christian_Cambon",
    "Agnès_Canayer",
    "Michel_Canévet",
    "Vincent_Capo-Canellas",
    "Emmanuel_Capus",
    "Rémi_Cardon",
    "Jean-Noël_Cardoux",
    "Marie-Arlette_Carlotti",
    "Maryse_Carrère",
    "Alain_Cazabonne",
    "Samantha_Cazebonne",
    "Anne_Chain-Larché",
    "Patrick_Chaize",
    "Yan_Chantrel",
    "Pierre_Charon",
    "Daniel_Chasseing",
    "Alain_Chatillon",
    "Patrick_Chauvet",
    "Marie-Christine_Chauvin",
    "Guillaume_Chevrollier",
    "Marta_de_Cidrac",
    "Olivier_Cigolotti",
    "Laurence_Cohen",
    "Catherine_Conconne",
    "Hélène_Conway-Mouret",
    "Jean-Pierre_Corbisez",
    "Édouard_Courtial",
    "Thierry_Cozic",
    "Cécile_Cukierman",
    "Pierre_Cuypers",
    "Michel_Dagbert",
    "Ronan_Dantec",
    "Laure_Darcos",
    "Mathieu_Darnaud",
    "Marc-Philippe_Daubresse",
    "Jean-Pierre_Decool",
    "Vincent_Delahaye",
    "Nathalie_Delattre",
    "Bernard_Delcros",
    "Annie_Delmont-Koropoulis",
    "Patricia_Demas",
    "Stéphane_Demilly",
    "Michel_Dennemont",
    "Catherine_Deroche",
    "Chantal_Deseyne",
    "Yves_Détraigne",
    "Brigitte_Devésa",
    "Gilbert-Luc_Devinaz",
    "Catherine_Di_Folco",
    "Nassimah_Dindar",
    "Élisabeth_Doineau",
    "Philippe_Dominati",
    "Thomas_Dossus",
    "Sabine_Drexler",
    "Alain_Duffourg",
    "Catherine_Dumas",
    "Françoise_Dumont",
    "Laurent_Duplomb",
    "Jérôme_Durain",
    "Nicole_Duranton",
    "Vincent_Éblé",
    "Frédérique_Espagnac",
    "Dominique_Estrosi_Sassone",
    "Jacqueline_Eustache-Brinio",
    "Marie_Evrard",
    "Gilbert_Favreau",
    "Françoise_Férat",
    "Rémi_Féraud",
    "Corinne_Féret",
    "Jacques_Fernique",
    "Bernard_Fialaire",
    "Jean-Luc_Fichet",
    "Martine_Filleul",
    "Philippe_Folliot",
    "Bernard_Fournier",
    "Christophe-André_Frassa",
    "Pierre_Frogier",
    "Amel_Gacquerre",
    "Laurence_Garnier",
    "Joëlle_Garriaud-Maylam",
    "Françoise_Gatel",
    "André_Gattolin",
    "Fabien_Gay",
    "Fabien_Genet",
    "Frédérique_Gerbaud",
    "Hervé_Gillé",
    "Éric_Gold",
    "Guillaume_Gontard",
    "Béatrice_Gosselin",
    "Nathalie_Goulet",
    "Sylvie_Goy-Chavent",
    "Jean-Pierre_Grand",
    "Michelle_Gréaume",
    "Daniel_Gremillet",
    "Jacques_Grosperrin",
    "Pascale_Gruny",
    "Charles_Guené",
    "Daniel_Gueret",
    "Jean-Noël_Guérini",
    "Joël_Guerriau",
    "Jocelyne_Guidez",
    "Véronique_Guillotin",
    "André_Guiol",
    "Laurence_Harribey",
    "Abdallah_Hassani",
    "Nadège_Havet",
    "Ludovic_Haye",
    "Olivier_Henno",
    "Loïc_Hervé",
    "Christine_Herzog",
    "Jean_Hingray",
    "Jean-Michel_Houllegatte",
    "Alain_Houpert",
    "Jean-Raymond_Hugonet",
    "Jean-François_Husson",
    "Xavier_Iacovelli",
    "Corinne_Imbert",
    "Annick_Jacquemet",
    "Micheline_Jacques",
    "Olivier_Jacquin",
    "Jean-Marie_Janssens",
    "Victoire_Jasmin",
    "Éric_Jeansannetas",
    "Patrice_Joly",
    "Bernard_Jomier",
    "Else_Joseph",
    "Gisèle_Jourda",
    "Muriel_Jourda",
    "Alain_Joyandet",
    "Patrick_Kanner",
    "Roger_Karoutchi",
    "Claude_Kern",
    "Éric_Kerrouche",
    "Christian_Klinger",
    "Mikaele_Kulimoetoke",
    "Marie-Pierre_de_La_Gontrie",
    "Sonia_de_La_Provôté",
    "Joël_Labbé",
    "Laurent_Lafon",
    "Jean-Louis_Lagourgue",
    "Gérard_Lahellec",
    "Marc_Laménie",
    "Gérard_Larcher",
    "Florence_Lassarade",
    "Michel_Laugier",
    "Daniel_Laurent",
    "Pierre_Laurent",
    "Christine_Lavarde",
    "Jean-Yves_Leconte",
    "Antoine_Lefèvre",
    "Dominique_de_Legge",
    "Ronan_Le_Gleut",
    "Annie_Le_Houerou",
    "Jean-Baptiste_Lemoyne",
    "Jacques_Le_Nay",
    "Olivier_Léonhardt",
    "Henri_Leroy",
    "Stéphane_Le_Rudulier",
    "Valérie_Létard",
    "Pierre-Antoine_Levi",
    "Martin_Lévrier",
    "Brigitte_Lherbier",
    "Marie-Noëlle_Lienemann",
    "Anne-Catherine_Loisier",
    "Jean-François_Longeot",
    "Gérard_Longuet",
    "Vivette_Lopez",
    "Pierre_Louault",
    "Jean-Jacques_Lozach",
    "Monique_Lubin",
    "Victorin_Lurel",
    "Jacques-Bernard_Magner",
    "Viviane_Malet",
    "Claude_Malhuret",
    "Didier_Mandelli",
    "Alain_Marc",
    "Frédéric_Marchand",
    "Monique_de_Marco",
    "Didier_Marie",
    "Hervé_Marseille",
    "Pascal_Martin",
    "Jean_Louis_Masson",
    "Hervé_Maurey",
    "Pierre_Médevielle",
    "Thierry_Meignen",
    "Colette_Mélot",
    "Franck_Menonville",
    "Marie_Mercier",
    "Serge_Mérillou",
    "Michelle_Meunier",
    "Sébastien_Meurant",
    "Jean-Jacques_Michau",
    "Brigitte_Micouleau",
    "Alain_Milon",
    "Jean-Marie_Mizzon",
    "Jean-Pierre_Moga",
    "Thani_Mohamed_Soilihi",
    "Marie-Pierre_Monier",
    "Franck_Montaugé",
    "Albéric_de_Montgolfier",
    "Catherine_Morin-Desailly",
    "Philippe_Mouiller",
    "Laurence_Muller-Bronn",
    "Philippe_Nachbar",
    "Louis-Jean_de_Nicolaÿ",
    "Sylviane_Noël",
    "Claude_Nougein",
    "Pierre_Ouzoulias",
    "Olivier_Paccaud",
    "Guylène_Pantel",
    "Jean-Jacques_Panunzi",
    "Vanina_Paoli-Gagin",
    "Paul_Toussaint_Parigi",
    "Georges_Patient",
    "François_Patriat",
    "Philippe_Paul",
    "Cyril_Pellevat",
    "Philippe_Pemezec",
    "Cédric_Perrin",
    "Évelyne_Perrot",
    "Annick_Petrus",
    "Marie-Laure_Phinera-Horth",
    "Stéphane_Piednoir",
    "Sebastien_Pla",
    "Kristina_Pluchet",
    "Gérard_Poadja",
    "Rémy_Pointereau",
    "Raymonde_Poncet_Monge",
    "Émilienne_Poumirol",
    "Angèle_Préville",
    "Sophie_Primas",
    "Jean-Paul_Prince",
    "Catherine_Procaccia",
    "Frédérique_Puissat",
    "Daphné_Ract-Madoux",
    "Isabelle_Raimond-Pavero",
    "Didier_Rambaud",
    "Jean-François_Rapin",
    "Stéphane_Ravier",
    "Claude_Raynal",
    "Christian_Redon-Sarrazy",
    "Damien_Regnard",
    "André_Reichardt",
    "Évelyne_Renaud-Garabedian",
    "Jean-Claude_Requier",
    "Bruno_Retailleau",
    "Alain_Richard",
    "Marie-Pierre_Richer",
    "Olivier_Rietmann",
    "Sylvie_Robert",
    "Gilbert_Roger",
    "Teva_Rohfritsch",
    "Bruno_Rojouan",
    "Laurence_Rossignol",
    "Jean-Yves_Roux",
    "Denise_Saint-Pé",
    "Daniel_Salmon",
    "Hugues_Saury",
    "Stéphane_Sautarel",
    "René-Paul_Savary",
    "Michel_Savin",
    "Pascal_Savoldelli",
    "Elsa_Schalck",
    "Patricia_Schillinger",
    "Vincent_Segouin",
    "Bruno_Sido",
    "Jean_Sol",
    "Nadia_Sollogoub",
    "Laurent_Somon",
    "Lucien_Stanzione",
    "Jean-Pierre_Sueur",
    "Philippe_Tabarot",
    "Sophie_Taillé-Polian",
    "Rachid_Temal",
    "Lana_Tetuanui",
    "Dominique_Théophile",
    "Claudine_Thomas",
    "Jean-Claude_Tissot",
    "Jean-Marc_Todeschini",
    "Mickaël_Vallet",
    "André_Vallini",
    "Sabine_Van_Heghe",
    "Jean-Marie_Vanlerenberghe",
    "Marie-Claude_Varaillas",
    "Yannick_Vaugrenard",
    "Anne_Ventalon",
    "Dominique_Vérien",
    "Sylvie_Vermeillet",
    "Pierre-Jean_Verzelen",
    "Cédric_Vial",
    "Jean_Pierre_Vogel",
    "Mélanie_Vogel",
    "Dany_Wattebled"
]

import shutil
source_folder = '/content/drive/MyDrive/French_Outpu_22_9/original_data'

# Dossier a les fichier CSV apres filtre
destination_folder = '/content/drive/MyDrive/French_Outpu_22_9/filter_data'
count = 1
os.makedirs(destination_folder, exist_ok=True)

for filename in os.listdir(source_folder):
    if filename.endswith('.csv'):
        file_name_without_extension = os.path.splitext(filename)[0]
        if file_name_without_extension in formatted_names:
            source_file_path = os.path.join(source_folder, filename)
            destination_file_path = os.path.join(destination_folder, filename)
            shutil.copy2(source_file_path, destination_file_path)
            print(f"Đã sao chép file: {filename}")
